# Sok format
We would follow the formatting convention used for "Sokoban YASC"

## Legend

| Symbol                         | Meaning                              |
| ------------------------------ | ------------------------------------ |
| `#`                            | Wall                                 |
| `p`                            | Pusher                               |
| `P`                            | Pusher on goal square               |
| `b`                            | Box                                  |
| `B`                            | Box on goal square                  |
| `.`                            | Goal square                          |
| ` `                            | Floor                                |
| `-`                            | Floor                                |
| `_`                            | Floor                                |

## Remarks

- The first and last non-empty square in each row must be a wall or a box on a goal.
- An empty interior row must be written with at least one `-` or `_`.

In [1]:
using Printf

mutable struct Sokoban
    width::Int
    height::Int
    player::Tuple{Int, Int}  # Player position (row, col)
    boxes::Vector{Tuple{Int, Int}}  # List of box positions
    goals::Vector{Tuple{Int, Int}}  # List of goal positions
    walls::Vector{Tuple{Int, Int}}  # List of wall positions
end

In [9]:
function extract_levels(file_path::String)::Vector{String}
    file = open(file_path, "r")
    lines = readlines(file)
    close(file)

    levels = String[]
    current_level = []
    recording = false

    for line in lines
        if occursin(r"^\s*#", line)  # Detect level lines starting with "#"
            push!(current_level, line)
            recording = true
        elseif recording
            if isempty(line)  # Empty line indicates the end of a level
                push!(levels, join(current_level, "\n"))
                current_level = []
                recording = false
            end
        end
    end

    if !isempty(current_level)  # Capture last level if no empty line at the end
        push!(levels, join(current_level, "\n"))
    end

    return levels
end

function save_levels(levels::Vector{String}, output_dir::String)
    mkpath(output_dir)  # Ensure output directory exists
    for (i, level) in enumerate(levels)
        file_path = joinpath(output_dir, "level_$i.txt")
        open(file_path, "w") do f
            write(f, level)
        end
    end
end

Extracted 90 levels and saved to levels_extracted


In [16]:
#extracting levels from file and store in separate txt

file_path = "Xsokoban_90.xsb"  # Path to the file containing levels
#output_dir = "Xsokoban_90"  # Directory to store extracted levels

levels = extract_levels(file_path)
#save_levels(levels, output_dir)

println("Extracted $(length(levels)) levels and saved to $output_dir")

Extracted 90 levels and saved to Xsokoban_90


In [169]:
# Parse the level into a 2D array (grid)
function parse_level_to_grid(level_str::String)
    return [collect(replace(line, '@' => 'p', '$' => 'b', '+' => 'P', '*' => 'b')) for line in split(level_str, "\n") if !isempty(line)]
end

parse_level_to_grid (generic function with 1 method)

In [170]:
# Render the grid
function render_grid(grid::Vector{Vector{Char}})
    for row in grid
        println(join(row))
    end
end

render_grid (generic function with 1 method)

In [171]:
# Find player position
function find_player_position(grid::Vector{Vector{Char}})
    for (i, row) in enumerate(grid)
        for (j, cell) in enumerate(row)
            if cell == 'p' || cell == 'P'
                return (i, j)  # Return row, column of the player
            end
        end
    end
    error("Player not found in the grid!")
end

find_player_position (generic function with 1 method)

In [207]:
# Move the player on the grid
function move_player!(grid::Vector{Vector{Char}}, direction::String)
    # Get player position
    row, col = find_player_position(grid)
    
    # Determine the new player position based on direction
    new_row, new_col = row, col
    
    if direction == "up" && row > 1
        new_row -= 1
    elseif direction == "down" && row < length(grid)
        new_row += 1
    elseif direction == "left" && col > 1
        new_col -= 1
    elseif direction == "right" && col < maximum(length, grid)  # Check right boundary correctly
        new_col += 1
    else
        println("Invalid move!, out of bound")
        return (row, col)  # Return the original position if out of bounds
    end
    
    # Get the character at the new position
    new_pos = grid[new_row][new_col]
    
    if new_pos == ' ' || new_pos == '-' || new_pos == '_'  # Empty space or floor
        # Move player to the empty space
        if grid[row][col] == 'p'  # Player was on the floor
            grid[new_row][new_col] = 'p'  # Move to new floor position
            grid[row][col] = ' '  # Clear old player position
        elseif grid[row][col] == 'P'  # Player was on goal
            grid[new_row][new_col] = 'p'  # Move to floor after leaving goal
            grid[row][col] = '.'  # Clear old player position
        end
        
        return (new_row, new_col)
    
    elseif new_pos == '.'  # Goal square
        # Move player to the goal square
        if grid[row][col] == 'p'  # Player was on floor
            grid[new_row][new_col] = 'P'  # Move to goal square
            grid[row][col] = ' '  # Clear old player position
        elseif grid[row][col] == 'P'  # Player already on goal
            grid[new_row][new_col] = 'P'  # Player stays on goal
            grid[row][col] = '.'  # Clear old player position
        end
        
        return (new_row, new_col)
    
    elseif new_pos == 'b' || new_pos == 'B'  # Box
        # Try to move the box
        if can_move_box(grid, new_row, new_col, direction)
            # Move the box to the target or empty spot
            if new_pos == 'B'  # Box was on the goal square
                grid[new_row][new_col] = 'P'  # Keep box on goal square
            else
                grid[new_row][new_col] = 'p'  # Box moves to floor
            end
            grid[row][col] = ' '  # Clear old player position
            return (new_row, new_col)
        else
            println("Cannot move box!")
            return (row, col)  # Return player position if move not possible
        end
    else
        println("Invalid move! You are hitting the wall")
        return (row, col)  # Return the original position if invalid move
    end
end

move_player! (generic function with 1 method)

In [208]:
# Function to check if a box can be moved
function can_move_box(grid::Vector{Vector{Char}}, row::Int, col::Int, direction::String)
    # Determine where the box will move based on the direction
    if direction == "up"
        target_row, target_col = row - 1, col
    elseif direction == "down"
        target_row, target_col = row + 1, col
    elseif direction == "left"
        target_row, target_col = row, col - 1
    elseif direction == "right"
        target_row, target_col = row, col + 1
    else
        return false  # Invalid direction
    end
    
    # Check if the target position is valid (empty space or goal)
    if grid[target_row][target_col] == ' ' || grid[target_row][target_col] == '-' || grid[target_row][target_col] == '_'
        grid[target_row][target_col] = 'b'
        return true  # Box can be moved to an empty floor space
    elseif grid[target_row][target_col] == '.'
        grid[target_row][target_col] = 'B'
        return true  # Box can be moved to a goal square
    else
        return false  # Box cannot be moved to walls or other boxes
    end
end

can_move_box (generic function with 1 method)

In [209]:
# Check if the game is won (all boxes are on target positions)
function check_win(grid::Vector{Vector{Char}})
    for row in grid
        for cell in row
            if cell == 'b'  # If there's still a box that isn't on the target, the game is not won
                return false
            end
        end
    end
    return true  # If no boxes are left, the game is won
end

check_win (generic function with 1 method)

In [228]:
function play_game(level::String)
    grid = parse_level_to_grid(level)
    original_grid = deepcopy(grid)  # Store the original grid for reset purposes
    grid_history = [deepcopy(grid)]  # History stack for undo
    render_grid(grid)
    
    # Keep playing until the player wins
    while !check_win(grid)
        print("Move (up/down/left/right or single letter , or reset or undo): ")
        IJulia.clear_output(true)
        move = readline()  # Wait for the player's input
        
        # Trim whitespace and convert to lowercase for case-insensitivity
        move = lowercase(strip(move))

        move_dir = ""

        if move == "reset"
            grid = deepcopy(original_grid)  # Reset the grid
            grid_history = [deepcopy(grid)]  # Clear history and reset it
            println("Game reset!")
        elseif move == "undo" && length(grid_history) > 1
            pop!(grid_history)  # Remove the most recent state
            grid = grid_history[end]  # Revert to the last state
            println("Undo successful!")
        # Map the input to the corresponding direction
        elseif move == "u" || move == "up"
            move_dir = "up"
        elseif move == "d" || move == "down"
            move_dir = "down"
        elseif move == "l" || move == "left"
            move_dir = "left"
        elseif move == "r" || move == "right"
            move_dir = "right"
        else
            println("Invalid move! Please enter 'up', 'down', 'left', 'right' or the first letters.")
            continue  # Skip the rest of the loop if input is invalid
        end
        
        # Move the player and add the new state to history
        if move_dir != ""  # Valid direction
            move_player!(grid, move_dir)
        end
        render_grid(grid)
        push!(grid_history, deepcopy(grid))  # Save the current grid state to history           
    end
    
    println("You win!")
end

play_game (generic function with 1 method)

In [229]:
# Start the game
play_game(test)

LoadError: InterruptException:

In [183]:
test=raw"########
#      #
#@ $   #
#   $  #
# . .  #
########"

"########\n#      #\n#@ \$   #\n#   \$  #\n# . .  #\n########"